# ETL Project 

---
This project's purpose is to work with a partner to find multiple datasets or data sources and load them together for future analysis. We (Sarah & Paul) are interested in health patterns across the US, so this interest guided our search. Notes are included as we go.


In [55]:
# import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import pymysql
from sqlalchemy import create_engine
import sys
sys.path.append('../..')
from config import local_sql_dbuser, local_sql_dbpwd 
pymysql.install_as_MySQLdb() 

---
## Import "Welcome" image from web scraping to welcome you to our project
This is purely just for fun, this is not data loaded into our database.

In [56]:
#scraping a web page to bring in an image to welcome you to our project
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

#found the image on google and it linked to this medium article
url = 'https://medium.ip.sx/meet-our-new-project-manager-ovidiu-rauca-263e4d6a6cfc'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [57]:
image = soup.find("img", class_="progressiveMedia-image")["src"]
img_url = image
featured_image_url = img_url

In [58]:
# Use the requests library to download and save the image from the `img_url` above
import requests
import shutil
response = requests.get(img_url, stream=True)
with open('img.jpg', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
    
# Display the image with IPython.display
from IPython.display import Image
Image(url='img.jpg')

---
## Load Mortality Data
We found data on Mortality Rates in the United States from CDC.gov: https://wonder.cdc.gov/controller/datarequest/D140
We decided to export this dataset based on:
* County code
* ICD Chapter (type of death)
* death count
* population of county

The data exported as a txt file. Below I'm reading in the file to pandas like it is a csv, using "\t" to indicate the tab delimited separations between the data cells.

In [60]:
data = pd.read_csv('Resources/Deaths.txt', sep="\t")
data.head()

,Notes,County,County Code,ICD Chapter,ICD Chapter Code,Deaths,Population,Crude Rate,% of Total Deaths
0,NaN,"Autauga County, AL",1001.0,Certain infectious and parasitic diseases,A00-B99,211.0,918492.0,23.0,0.0%
1,NaN,"Autauga County, AL",1001.0,Neoplasms,C00-D48,1771.0,918492.0,192.8,0.0%
2,NaN,"Autauga County, AL",1001.0,Diseases of the blood and blood-forming organs...,D50-D89,44.0,918492.0,4.8,0.0%
3,NaN,"Autauga County, AL",1001.0,"Endocrine, nutritional and metabolic diseases",E00-E88,354.0,918492.0,38.5,0.0%
4,NaN,"Autauga County, AL",1001.0,Mental and behavioural disorders,F01-F99,218.0,918492.0,23.7,0.0%


In [61]:
# drop notes column
data.drop(columns=["Notes"], inplace=True)
data.head()

,County,County Code,ICD Chapter,ICD Chapter Code,Deaths,Population,Crude Rate,% of Total Deaths
0,"Autauga County, AL",1001.0,Certain infectious and parasitic diseases,A00-B99,211.0,918492.0,23.0,0.0%
1,"Autauga County, AL",1001.0,Neoplasms,C00-D48,1771.0,918492.0,192.8,0.0%
2,"Autauga County, AL",1001.0,Diseases of the blood and blood-forming organs...,D50-D89,44.0,918492.0,4.8,0.0%
3,"Autauga County, AL",1001.0,"Endocrine, nutritional and metabolic diseases",E00-E88,354.0,918492.0,38.5,0.0%
4,"Autauga County, AL",1001.0,Mental and behavioural disorders,F01-F99,218.0,918492.0,23.7,0.0%


## Load Mortality Data into SQL

Insert local connections then read in data to SQL.

In [63]:
# Connect to local database'
rds_connection_string = f"{local_sql_dbuser}:{local_sql_dbpwd}@localhost:3306/etl_mortality"
engine = create_engine(f'mysql://{rds_connection_string}')

In [64]:
#add to db
data.to_sql(name='mortality_data', con=engine, if_exists='append', index=False)

# check for table names
engine.table_names()

['fips_df', 'mortality_data']

In [65]:
# Confirm data has been added by querying the mortality_data table
pd.read_sql_query('select * from mortality_data', con=engine).head()

,County,County Code,ICD Chapter,ICD Chapter Code,Deaths,Population,Crude Rate,% of Total Deaths
0,"Autauga County, AL",1001.0,Certain infectious and parasitic diseases,A00-B99,211.0,918492.0,23.0,0.0%
1,"Autauga County, AL",1001.0,Neoplasms,C00-D48,1771.0,918492.0,192.8,0.0%
2,"Autauga County, AL",1001.0,Diseases of the blood and blood-forming organs...,D50-D89,44.0,918492.0,4.8,0.0%
3,"Autauga County, AL",1001.0,"Endocrine, nutritional and metabolic diseases",E00-E88,354.0,918492.0,38.5,0.0%
4,"Autauga County, AL",1001.0,Mental and behavioural disorders,F01-F99,218.0,918492.0,23.7,0.0%


## Load in Census Block Data
A larger helpful reference tool to add to our database.

In [66]:
# import csv into a dataframe
geo_df = pd.read_csv("Resources/census_block.csv", encoding='latin 1')

# show me what you got
geo_df.head()

,GIDBG,State,State_name,County,County_name,Tract,Block_Group
0,1.001020e+10,1,Alabama,1,Autauga County,20100,1
1,1.001020e+10,1,Alabama,1,Autauga County,20100,2
2,1.001020e+10,1,Alabama,1,Autauga County,20200,1
3,1.001020e+10,1,Alabama,1,Autauga County,20200,2
4,1.001020e+10,1,Alabama,1,Autauga County,20300,1


In [67]:
# add preceding zeros to the columns in order to create a 12 digits census block group code
geo_df['State'] = geo_df['State'].apply(lambda x: '{0:0>2}'.format(x))
geo_df['County'] = geo_df['County'].apply(lambda x: '{0:0>3}'.format(x))
geo_df['Tract'] = geo_df['Tract'].apply(lambda x: '{0:0>6}'.format(x))

In [68]:
# create census block group
geo_df['Census_Block_Group'] = geo_df['State'].astype(str) + geo_df['County'].astype(str) + geo_df['Tract'].astype(str) + geo_df['Block_Group'].astype(str)

# show me what you got
geo_df.head()

,GIDBG,State,State_name,County,County_name,Tract,Block_Group,Census_Block_Group
0,1.001020e+10,01,Alabama,001,Autauga County,020100,1,010010201001
1,1.001020e+10,01,Alabama,001,Autauga County,020100,2,010010201002
2,1.001020e+10,01,Alabama,001,Autauga County,020200,1,010010202001
3,1.001020e+10,01,Alabama,001,Autauga County,020200,2,010010202002
4,1.001020e+10,01,Alabama,001,Autauga County,020300,1,010010203001


## Add in FIPS code
FIPS (Federal Information Processing Standards) are a set of standards that describe document processing, encryption algorithms and other information technology standards that are used in government databases and referenced in the mortality data we just pulled in.

In order for the Census block group data to be able to reference the other datasets in our database, we separated FIPS data here.

In [69]:
# create 5 digit FIPS code by adding state and county codes together
geo_df['FIPS'] = geo_df['State'].astype(str) + geo_df['County'].astype(str)

# show me what you got
geo_df.head()

,GIDBG,State,State_name,County,County_name,Tract,Block_Group,Census_Block_Group,FIPS
0,1.001020e+10,01,Alabama,001,Autauga County,020100,1,010010201001,01001
1,1.001020e+10,01,Alabama,001,Autauga County,020100,2,010010201002,01001
2,1.001020e+10,01,Alabama,001,Autauga County,020200,1,010010202001,01001
3,1.001020e+10,01,Alabama,001,Autauga County,020200,2,010010202002,01001
4,1.001020e+10,01,Alabama,001,Autauga County,020300,1,010010203001,01001


In [70]:
# create 11 digit census tract
geo_df['Census_Tract'] = geo_df['State'].astype(str) + geo_df['County'].astype(str) + geo_df['Tract'].astype(str)
geo_df.head()

,GIDBG,State,State_name,County,County_name,Tract,Block_Group,Census_Block_Group,FIPS,Census_Tract
0,1.001020e+10,01,Alabama,001,Autauga County,020100,1,010010201001,01001,01001020100
1,1.001020e+10,01,Alabama,001,Autauga County,020100,2,010010201002,01001,01001020100
2,1.001020e+10,01,Alabama,001,Autauga County,020200,1,010010202001,01001,01001020200
3,1.001020e+10,01,Alabama,001,Autauga County,020200,2,010010202002,01001,01001020200
4,1.001020e+10,01,Alabama,001,Autauga County,020300,1,010010203001,01001,01001020300


In [71]:
# create columns with no preceding zeros (some data sources omit the preceding zero with FIPS, CBG, and Census Tract)
geo_df['No_Zero_FIPS'] = [x.lstrip("0") for x in geo_df['FIPS']]
geo_df['No_Zero_CBG'] = [x.lstrip("0") for x in geo_df['Census_Block_Group']]
geo_df['No_Zero_Census_Tract'] = [x.lstrip("0") for x in geo_df['Census_Tract']]

In [72]:
# replace Puerto Rico Commonwewealth with Puerto Rico for a later merge
geo_df.replace({'Puerto Rico Commonwealth': 'Puerto Rico'})

,GIDBG,State,State_name,County,County_name,Tract,Block_Group,Census_Block_Group,FIPS,Census_Tract,No_Zero_FIPS,No_Zero_CBG,No_Zero_Census_Tract
0,1.001020e+10,01,Alabama,001,Autauga County,020100,1,010010201001,01001,01001020100,1001,10010201001,1001020100
1,1.001020e+10,01,Alabama,001,Autauga County,020100,2,010010201002,01001,01001020100,1001,10010201002,1001020100
2,1.001020e+10,01,Alabama,001,Autauga County,020200,1,010010202001,01001,01001020200,1001,10010202001,1001020200
3,1.001020e+10,01,Alabama,001,Autauga County,020200,2,010010202002,01001,01001020200,1001,10010202002,1001020200
4,1.001020e+10,01,Alabama,001,Autauga County,020300,1,010010203001,01001,01001020300,1001,10010203001,1001020300
5,1.001020e+10,01,Alabama,001,Autauga County,020300,2,010010203002,01001,01001020300,1001,10010203002,1001020300
6,1.001020e+10,01,Alabama,001,Autauga County,020400,1,010010204001,01001,01001020400,1001,10010204001,1001020400
7,1.001020e+10,01,Alabama,001,Autauga County,020400,2,010010204002,01001,01001020400,1001,10010204002,1001020400
8,1.001020e+10,01,Alabama,001,Autauga County,020400,3,010010204003,01001,01001020400,1001,10010204003,1001020400
9,1.001020e+10,01,Alabama,001,Autauga County,020400,4,010010204004,01001,01001020400,1001,10010204004,1001020400


In [73]:
# clean up and re-order columns
geo_df = geo_df[['State', 'State_name', 'County', 'County_name',
                  'FIPS', 'No_Zero_FIPS', 'Tract', 'Census_Tract',
                  'No_Zero_Census_Tract', 'Census_Block_Group', 'No_Zero_CBG',
                  'Block_Group']]

## Web Scraping for state abbreviations

In [39]:
# URL to scrape to add state abbreviations
url = 'https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations'

# scrape the URL
tables = pd.read_html(url)

#locate the correct table and set it to a dataframe
df_state = tables[0]

In [41]:
# remove the first rows because of unnecessary data
df_state = df_state.drop(df_state.index[0:12])
df_state.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
12,Alabama,State,US-AL,AL,01,AL,AL,Ala.,Ala.,NaN,NaN,NaN,NaN,NaN,NaN
13,Alaska,State,US-AK,AK,02,AK,AK,Alaska,Alaska,Alas.,NaN,NaN,NaN,NaN,NaN
14,Arizona,State,US-AZ,AZ,04,AZ,AZ,Ariz.,Ariz.,Az.,NaN,NaN,NaN,NaN,NaN
15,Arkansas,State,US-AR,AR,05,AR,AR,Ark.,Ark.,NaN,NaN,NaN,NaN,NaN,NaN
16,California,State,US-CA,CA,06,CA,CF,Calif.,Calif.,"Ca., Cal.",NaN,NaN,NaN,NaN,NaN


In [42]:
# rename numbered columns to match the table columns from the wikipedia
df_state.rename(columns={ df_state.columns[0]: "State",
                        df_state.columns[2]: "ISO",
                        df_state.columns[3]: "ANSI",
                        df_state.columns[5]: "USPS",
                        df_state.columns[6]: "USCG",
                        df_state.columns[7]: "GPO",
                        df_state.columns[8]: "AP",}, inplace=True)
df_state.head()

,State,1,ISO,ANSI,4,USPS,USCG,GPO,AP,9,10,11,12,13,14
12,Alabama,State,US-AL,AL,01,AL,AL,Ala.,Ala.,NaN,NaN,NaN,NaN,NaN,NaN
13,Alaska,State,US-AK,AK,02,AK,AK,Alaska,Alaska,Alas.,NaN,NaN,NaN,NaN,NaN
14,Arizona,State,US-AZ,AZ,04,AZ,AZ,Ariz.,Ariz.,Az.,NaN,NaN,NaN,NaN,NaN
15,Arkansas,State,US-AR,AR,05,AR,AR,Ark.,Ark.,NaN,NaN,NaN,NaN,NaN,NaN
16,California,State,US-CA,CA,06,CA,CF,Calif.,Calif.,"Ca., Cal.",NaN,NaN,NaN,NaN,NaN


In [43]:
# mass remove columns from dataframe with unnecessary or null data & reset the index
cols = [1,4,9,10,11,12,13,14]
df_state.drop(df_state.columns[cols],axis=1,inplace=True)

df_state = df_state.reset_index(drop=True)

In [44]:
# remove rows for non-states and reset the index so that puerto rico doesn't look out of place
df_state = df_state.drop(df_state.index[55:85])
df_state = df_state.drop(df_state.index[51:54])
df_state = df_state.reset_index(drop=True)

df_state.head()

,State,ISO,ANSI,USPS,USCG,GPO,AP
0,Alabama,US-AL,AL,AL,AL,Ala.,Ala.
1,Alaska,US-AK,AK,AK,AK,Alaska,Alaska
2,Arizona,US-AZ,AZ,AZ,AZ,Ariz.,Ariz.
3,Arkansas,US-AR,AR,AR,AR,Ark.,Ark.
4,California,US-CA,CA,CA,CF,Calif.,Calif.


In [45]:
# merge geo_df with state_df, call it fips_df
fips_df = geo_df.merge(df_state, left_on='State_name', right_on='State')
fips_df.head()

,State_x,State_name,County,County_name,FIPS,No_Zero_FIPS,Tract,Census_Tract,No_Zero_Census_Tract,Census_Block_Group,No_Zero_CBG,Block_Group,State_y,ISO,ANSI,USPS,USCG,GPO,AP
0,01,Alabama,001,Autauga County,01001,1001,020100,01001020100,1001020100,010010201001,10010201001,1,Alabama,US-AL,AL,AL,AL,Ala.,Ala.
1,01,Alabama,001,Autauga County,01001,1001,020100,01001020100,1001020100,010010201002,10010201002,2,Alabama,US-AL,AL,AL,AL,Ala.,Ala.
2,01,Alabama,001,Autauga County,01001,1001,020200,01001020200,1001020200,010010202001,10010202001,1,Alabama,US-AL,AL,AL,AL,Ala.,Ala.
3,01,Alabama,001,Autauga County,01001,1001,020200,01001020200,1001020200,010010202002,10010202002,2,Alabama,US-AL,AL,AL,AL,Ala.,Ala.
4,01,Alabama,001,Autauga County,01001,1001,020300,01001020300,1001020300,010010203001,10010203001,1,Alabama,US-AL,AL,AL,AL,Ala.,Ala.


In [47]:
# rename original state_x as state and dropping the duplicate state_y column
fips_df = fips_df.rename(columns={'State_x': 'State'})
fips_df = fips_df.drop(['State_y'], axis=1)

fips_df.head()

,State,State_name,County,County_name,FIPS,No_Zero_FIPS,Tract,Census_Tract,No_Zero_Census_Tract,Census_Block_Group,No_Zero_CBG,Block_Group,ISO,ANSI,USPS,USCG,GPO,AP
0,01,Alabama,001,Autauga County,01001,1001,020100,01001020100,1001020100,010010201001,10010201001,1,US-AL,AL,AL,AL,Ala.,Ala.
1,01,Alabama,001,Autauga County,01001,1001,020100,01001020100,1001020100,010010201002,10010201002,2,US-AL,AL,AL,AL,Ala.,Ala.
2,01,Alabama,001,Autauga County,01001,1001,020200,01001020200,1001020200,010010202001,10010202001,1,US-AL,AL,AL,AL,Ala.,Ala.
3,01,Alabama,001,Autauga County,01001,1001,020200,01001020200,1001020200,010010202002,10010202002,2,US-AL,AL,AL,AL,Ala.,Ala.
4,01,Alabama,001,Autauga County,01001,1001,020300,01001020300,1001020300,010010203001,10010203001,1,US-AL,AL,AL,AL,Ala.,Ala.


## Add the FIPS dataframe to MongoDB
Now the two datasets are available in the Mongo database.

In [50]:
#add to db
data.to_sql(name='fips_df', con=engine, if_exists='append', index=False)

# Confirm data has been added by querying the fips_df table
pd.read_sql_query('select * from fips_df', con=engine).head()

,County,County Code,ICD Chapter,ICD Chapter Code,Deaths,Population,Crude Rate,% of Total Deaths
0,"Autauga County, AL",1001.0,Certain infectious and parasitic diseases,A00-B99,211.0,918492.0,23.0,0.0%
1,"Autauga County, AL",1001.0,Neoplasms,C00-D48,1771.0,918492.0,192.8,0.0%
2,"Autauga County, AL",1001.0,Diseases of the blood and blood-forming organs...,D50-D89,44.0,918492.0,4.8,0.0%
3,"Autauga County, AL",1001.0,"Endocrine, nutritional and metabolic diseases",E00-E88,354.0,918492.0,38.5,0.0%
4,"Autauga County, AL",1001.0,Mental and behavioural disorders,F01-F99,218.0,918492.0,23.7,0.0%


In [51]:
# check for both table names
engine.table_names()

['fips_df', 'mortality_data']